# Following [SpaGE Tutorial](https://github.com/tabdelaal/SpaGE/blob/master/SpaGE_Tutorial.ipynb)

Integration of *osmFISH* spatial data with the *AllenSSp* scRNA-seq data

In [1]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt

import scipy
import scipy.stats as st
from scipy import linalg
from scipy import sparse as sp
import pickle
from sklearn.metrics.pairwise import euclidean_distances

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/Volumes/LaCie/school/combine_lab/SpaGE/SpaGE_Datasets/'

Load and preprocess the AllenSSp dataset

In [ ]:
RNA_data = pd.read_csv(path + 'scRNAseq/Allen_SSp/SSp_exons_matrix.csv',header=0,index_col=0,sep=',')

# filter lowely expressed genes
Genes_count = np.sum(RNA_data > 0, axis=1)
RNA_data = RNA_data.loc[Genes_count >=10,:] # filter out genes expressed in <10 cells
del Genes_count

Random select a gene for testing at very end (will perform cross validation on the training set)

In [ ]:
test_gene = RNA_data.sample(axis=0, random_state=42)
RNA_data_train = RNA_data.drop(test_gene.index) # remove gene from training data

# TODO: k-fold CV and LOOCV
see: https://www.thepythoncode.com/article/kfold-cross-validation-using-sklearn-in-python

In [ ]:
# Normalization
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1) # divide counts within each cell by the total number of transcripts within that cell, scale by 10^6 and log1p transformed.

RNA_ref_norm = RNA_data_train.apply(Log_Norm_cpm,axis=0)

In [ ]:
# standardize gene expression 
RNA_ref = st.zscore(RNA_ref_norm)
training_mean = scipy.mean(RNA_ref_norm, axis=0)
training_std = st.tstd(RNA_ref_norm, axis=0)

# normalize test set using training mean and std
RNA_test = (test_gene - training_mean) / training_std

# import spatial data

In [ ]:
ds = loompy.connect(path + 'Spatial/osmFISH/osmFISH_SScortex_mouse_all_cells.loom')
FISH_Genes = ds.ra['Gene']   
colAtr = ds.ca.keys()

df = pd.DataFrame()
for i in colAtr:
    df[i] = ds.ca[i]

osmFISH_meta = df.iloc[np.where(df.Valid == 1)[0], :]
osmFISH_data = ds[:,:]
osmFISH_data = osmFISH_data[:,np.where(df.Valid == 1)[0]]
osmFISH_data = pd.DataFrame(data= osmFISH_data, index= FISH_Genes)

del ds, colAtr, i, df, FISH_Genes

# Select cortical regions only to match the AllenSSp dataset
Cortex_Regions = ['Layer 2-3 lateral', 'Layer 2-3 medial', 'Layer 3-4', 
                  'Layer 4','Layer 5', 'Layer 6', 'Pia Layer 1']
Cortical = np.stack(i in Cortex_Regions for i in osmFISH_meta.Region)

osmFISH_meta = osmFISH_meta.iloc[Cortical,:]
osmFISH_data = osmFISH_data.iloc[:,Cortical]
del Cortex_Regions,Cortical

In [ ]:
osmFISH_meta

In [ ]:
# Normalization
cell_count = np.sum(osmFISH_data,axis=0)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

osmFISH_data_norm = osmFISH_data.apply(Log_Norm_spatial,axis=0)

In [ ]:
# scale
ST = st.zscore(osmFISH_data_norm)
ST

# Get ddf(x) for all training samples

## Create adj mat (inspired by [GCNG](https://github.com/xiaoyeye/GCNG/blob/master/data_generation_interaction_ten_fold.py))


In [ ]:
cell_st = osmFISH_meta.loc[:,['CellID', 'X', 'Y']]
cell_st

In [ ]:
cell_st = cell_st.reset_index(drop=True) # reset index
cell_st

From [GCNG](https://github.com/xiaoyeye/GCNG/blob/2fba06bac64e97744a9a8d2a1505aa5c2117730c/data_generation_interaction_ten_fold.py#L19)

In [ ]:
#### from GCNG ####

# ############ the distribution of distance
# distance_list = []
# print ('calculating distance matrix, it takes a while')

# for j in range(cell_st.shape[0]):
#     for i in range(cell_st.shape[0]):
#         if i!=j:
#             distance_list.append(np.linalg.norm(cell_st.iloc[j][['X','Y']]-cell_st.iloc[i][['X','Y']]))

In [ ]:
# #save out file

np.save(path+'distance_array.npy',np.array(distance_list))

# np.savetxt("dist_list.csv", 
#            distance_list,
#            delimiter ="\t")

import saved distance array (instead of re-running since it takes a long time to compute)

In [ ]:
distance_array = np.load(path+'distance_array.npy')
print(distance_array.shape)
distance_array

from [spektral](https://github.com/danielegrattarola/spektral/blob/1dbdf9e919ff4606ee5375567a9b993687c95bcd/spektral/utils/convolution.py#L47) -- unable to import the package (received 'illegal hardware' error)

In [ ]:
### from spektral ###
def degree_power(A, k):
    r"""
    Computes \(\D^{k}\) from the given adjacency matrix. Useful for computing
    normalised Laplacian.
    :param A: rank 2 array or sparse matrix.
    :param k: exponent to which elevate the degree matrix.
    :return: if A is a dense array, a dense array; if A is sparse, a sparse
    matrix in DIA format.
    """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        degrees = np.power(np.array(A.sum(1)), k).ravel()
    degrees[np.isinf(degrees)] = 0.0
    if sp.issparse(A):
        D = sp.diags(degrees)
    else:
        D = np.diag(degrees)
    return D

def normalized_adjacency(A, symmetric=True):
    r"""
    Normalizes the given adjacency matrix using the degree matrix as either
    \(\D^{-1}\A\) or \(\D^{-1/2}\A\D^{-1/2}\) (symmetric normalization).
    :param A: rank 2 array or sparse matrix;
    :param symmetric: boolean, compute symmetric normalization;
    :return: the normalized adjacency matrix.
    
    motivation: Takes into account both number of neighbours and their own number of neighbours
    """
    if symmetric:
        normalized_D = degree_power(A, -0.5)
        return normalized_D.dot(A).dot(normalized_D)
    else:
        normalized_D = degree_power(A, -1.0)
        return normalized_D.dot(A)

In [ ]:
###try different distance threshold, so that on average, each cell has x neighbor cells, see Tab. S1 for results

# distance_array = np.array(distance_list)
for threshold in range(100,400,40): #[100,140,180,210,220,260]:#range (210,211):#(100,400,40):
    num_big = np.where(distance_array<threshold)[0].shape[0]
    print (threshold,num_big,str(num_big/(len(cell_st)*2)))
    distance_matrix_threshold_I_list = []
#     distance_matrix_threshold_W_list = []
    distance_matrix = euclidean_distances(cell_st[['X','Y']], cell_st[['X','Y']])
    distance_matrix_threshold_I = np.zeros(distance_matrix.shape)
#     distance_matrix_threshold_W = np.zeros(distance_matrix.shape)
    for i in range(distance_matrix_threshold_I.shape[0]):
        for j in range(distance_matrix_threshold_I.shape[1]):
            if distance_matrix[i,j] <= threshold and distance_matrix[i,j] > 0:
                distance_matrix_threshold_I[i,j] = 1
#                 distance_matrix_threshold_W[i,j] = distance_matrix[i,j]
    distance_matrix_threshold_I_list.append(distance_matrix_threshold_I)
#     distance_matrix_threshold_W_list.append(distance_matrix_threshold_W)
# whole_distance_matrix_threshold_I = scipy.linalg.block_diag(distance_matrix_threshold_I_list[0],
#                                                             distance_matrix_threshold_I_list[1],
#                                                             distance_matrix_threshold_I_list[2],
#                                                             distance_matrix_threshold_I_list[3],
#                                                             distance_matrix_threshold_I_list[4],
#                                                             distance_matrix_threshold_I_list[5],
#                                                             distance_matrix_threshold_I_list[6])
    ############### get normalized sparse adjacent matrix
    distance_matrix_threshold_I_N = normalized_adjacency(distance_matrix_threshold_I, symmetric=True)
    # distance_matrix_threshold_I_N = np.float32(whole_distance_matrix_threshold_I) ## do not normalize adjcent matrix
    distance_matrix_threshold_I_N = np.float32(distance_matrix_threshold_I)
    distance_matrix_threshold_I_N_crs = sp.csr_matrix(distance_matrix_threshold_I_N)
    with open(path+'adj_mat_crs_'+str(threshold)+'.pickle', 'wb') as fp:
        pickle.dump(distance_matrix_threshold_I_N_crs, fp)

Import saved Adjacency matrices:

In [40]:
import os 

A = []
# read in all pickle files
for i, file in enumerate([path for path in os.listdir(path) if path.endswith('pickle')]):
    print(i, file)
    for i in [path+file]:
        with open(i, 'rb') as filepath:
            while True:
                try:
                    A.append(pickle.load(filepath))
                except EOFError:
                    break    
                    
# uncompress Compressed Sparse Row matrix
adj_mat = []
for i in range(len(A)):
    adj_mat.append(A[i].todense())

# get number of counts for all connected cells of cell i
sum_neighb_cell = []
for file in range(len(adj_mat)):
     sum_neighb_cell.append([(adj_mat[file][i][np.where(adj_mat[file][i] != 0)].size) for i in range(len(adj_mat[file]))])

# get average cell neighbourhood by threshold
for i in range(len(adj_mat)):
    avg_cell_neib = sum(sum_neighb_cell[i])/len(adj_mat[0])
    print('average cell neighborhood for file', i,':', avg_cell_neib)

0 adj_mat_crs_100.pickle
1 adj_mat_crs_140.pickle
2 adj_mat_crs_180.pickle
3 adj_mat_crs_210.pickle
4 adj_mat_crs_220.pickle
5 adj_mat_crs_260.pickle
6 adj_mat_crs_300.pickle
7 adj_mat_crs_340.pickle
8 adj_mat_crs_380.pickle
9 adj_mat_crs_400.pickle
average cell neighborhood for file 0 : 0.15036710719530103
average cell neighborhood for file 1 : 0.34772393538913365
average cell neighborhood for file 2 : 0.6807635829662262
average cell neighborhood for file 3 : 0.9985315712187959
average cell neighborhood for file 4 : 1.1236417033773862
average cell neighborhood for file 5 : 1.7527165932452275
average cell neighborhood for file 6 : 2.4176211453744494
average cell neighborhood for file 7 : 3.2052863436123347
average cell neighborhood for file 8 : 4.108076358296622
average cell neighborhood for file 9 : 4.59265785609398


GNCG [supp](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02214-w#Sec16)

seqFISH+ avg cell neighbourhood for different distance threshold:
- 100: 1.2
- 140: 2.5 <-- threshold with the best validation performace
- 180: 4.0
- 220: 5.8

MERFISH:
- 130: 1.03 <-- threshold with best validation performance
- 150: 1.52
- 170: 2.05
- 200: 2.95
- 240: 4.25

Try thresholds:
- 220: 1.12
- 300: 2.42

In [41]:
A200 = adj_mat[4]
A300 = adj_mat[6]